In [ ]:
import pyvisa
rm = pyvisa.ResourceManager()

In [ ]:
sdg = rm.open_resource(ADDRESS)
print(sdg.query("*IDN?"))

In [ ]:
sdg.write("C1:OUTP OFF")
sdg.write("C1:BSWV FRQ, 2000")
sdg.write("C1:BSWV AMP, 19")
sdg.write("C1:BSWV WVTP, SQUARE")
waveInfo = sdg.query("C1:BSWV?")

while 1 < 2:
    if waveInfo[0] != "C":
        waveInfo = sdg.query("C1:BSWV?")
    else:
        break

In [ ]:
freqIndex = waveInfo.find("FRQ")
voltIndexStart = waveInfo.find("AMP")
voltIndexEnd = waveInfo.find("V,AMPVRMS")

if waveInfo[freqIndex + 4:freqIndex + 8] != "2000":
    print("WARNING: INCORRECT FREQUENCY, MUST BE 2 kHz")
    raise SystemExit
if float(waveInfo[voltIndexStart+4:voltIndexEnd]) > 20.0:
    sdg.write("C1:AMP 1")
    print("WARNING: VOLTAGE TOO HIGH. VOLTAGE SHOULD BE NO GREATER THAN 20 V")
    raise SystemExit